In [10]:
import pandas as pd, numpy as np, openai, json, os, ast
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from retry import retry

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

### Dados

In [11]:
path_database = "temp_gpt_4_1_nano_vecs.csv"

In [12]:
database = pd.read_csv(path_database)
display(database.head(3))
database.shape

,resumo_reclamacao,resumo_resposta,tipo_reclamacao,produto_servico,topicos,trechos_adequados,trechos_inadequados,classificacao_resposta,grau_atendimento,redirecionamento,...,resolved_class,X_TEXT,embeddings_BoW,embeddings_Tfidf,embeddings_GPT3small,embeddings_DistilBert,embeddings_Bert,embeddings_Roberta,embeddings_Nomic,embeddings_OLMo
0,"A linha telefônica está bloqueada, impedindo c...",A empresa forneceu uma carta com informações s...,bloqueio de linha,serviço de telefonia móvel,"['bloqueio', 'chamadas', 'recarga', 'atendimen...",['a carta resposta com todas as informações re...,"['esclarecimento sobre cancelamento de linha, ...",parcial,não atendeu,Sim,...,1,"[CLS] A linha telefônica está bloqueada, imped...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.009672009386122227, 0.020684223622083664, -...","[-0.2666554, 0.024081107, 0.060418017, 0.14040...","[-0.4314905, 0.1272904, 0.031113207, -0.058524...","[0.034455013, 0.10360511, 0.0701988, -0.026581...","[0.7068655, 0.9845152, -3.277108, -0.7251988, ...","[-0.29681036, -0.4949199, 0.011052388, -0.0530..."
1,A cliente reclama do aumento no valor do pacot...,A empresa encaminhou uma carta com informações...,reajuste de preço e qualidade de serviço,internet banda larga,"['preço', 'negociação', 'conexão', 'manutenção...",['encaminhou uma carta com informações detalha...,"['não abordou diretamente a reclamação', 'não ...",genérica,não atendeu,Sim,...,1,[CLS] A cliente reclama do aumento no valor do...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1629377878408...","[-0.018821867182850838, 0.00856255553662777, 0...","[-0.36968863, 0.07371264, 0.07006773, 0.157783...","[-0.4747764, 0.13763846, 0.07152168, 0.0004466...","[0.040735167, 0.093870886, 0.06586944, -0.0330...","[0.63843286, 1.3483149, -3.4392283, -0.6547453...","[-0.43023008, -0.5411823, 0.2056812, -0.107438..."
2,A cliente foi cobrada por um valor superior ao...,"A empresa informou que tentou contato, enviou ...",cobrança indevida,plano de telefonia móvel,"['cobrança', 'valor', 'serviço adicional', 'at...",['Estamos à disposição para quaisquer esclarec...,['Realizamos tentativa de contato no dia 20/04...,genérica,não atendeu,Sim,...,1,[CLS] A cliente foi cobrada por um valor super...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1645977876294...","[-0.017444251105189323, 0.0062181088142097, 0....","[-0.35199377, 0.00018714466, 0.121630065, 0.07...","[-0.5137048, 0.07159388, 0.08065288, -0.121106...","[0.03248304, 0.13106053, 0.08480598, -0.024330...","[0.585095, 1.6964599, -3.8000524, -0.62313926,...","[-0.44809398, -0.47027147, 0.043780386, -0.247..."


(7035, 26)

In [ ]:
emb_cols = [c for c in database.columns if "embeddings" in c]
vec_names = [c.replace("embeddings_", "") for c in emb_cols]

# Convertendo strings de volta para listas
for ec in emb_cols:
    print(f"Convertendo {ec} para lista...")
    database[ec] = [ast.literal_eval(x) for x in database[ec]]

Convertendo embeddings_BoW para lista...
Convertendo embeddings_Tfidf para lista...
Convertendo embeddings_GPT3small para lista...


### Processamento

In [ ]:
database.columns

In [ ]:
df_results_emb = database[["service_class", "company"]]
df_results_emb["embeddings"] = database["embeddings_GPT3small"]

In [ ]:
# Embeddings e rótulos
X = np.vstack(df_results_emb["embeddings"].values)
y = df_results_emb["service_class"].values
groups = df_results_emb["company"].values
companies = df_results_emb["company"].values

# Normalização
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Modelos
modelos = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel="linear", probability=True, random_state=42),
    "MLP": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
}

# Cross-validation por empresa
gkf = GroupKFold(n_splits=3)

# Armazenar métricas por empresa
resultados_empresa = []

for nome_modelo, modelo in modelos.items():
    print(f"\n🔍 Avaliando modelo: {nome_modelo}")
    
    for fold, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups=groups)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        empresas_test = companies[test_idx]

        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        for empresa in np.unique(empresas_test):
            mask = empresas_test == empresa
            y_true_empresa = y_test[mask]
            y_pred_empresa = y_pred[mask]

            # Calcular F1-score macro (ou micro/weighted, conforme o caso)
            try:
                f1 = f1_score(y_true_empresa, y_pred_empresa, average="macro", zero_division=0)
            except:
                f1 = np.nan

            print(f"Empresa: {empresa}, Fold: {fold + 1}, F1 Macro: {f1:.4f}")
            
            resultados_empresa.append({
                "modelo": nome_modelo,
                "empresa": empresa,
                "fold": fold + 1,
                "num_amostras": len(y_true_empresa),
                "f1_macro": f1
            })

# Agrupar e calcular média do F1-score por empresa e modelo
df_f1 = pd.DataFrame(resultados_empresa)
df_f1_agrupado = df_f1.groupby(["modelo", "empresa"]).agg(
    media_f1_macro=("f1_macro", "mean"),
    qtd_folds=("fold", "count"),
    total_amostras=("num_amostras", "sum")
).reset_index()